In [2]:
# Functions
def pad_and_resize(image, target_size=(224, 224)):
    width, height = image.size
    longer_side = max(width, height)
    horizontal_padding = (longer_side - width) // 2
    vertical_padding = (longer_side - height) // 2

    padded_image = Image.new(image.mode, (longer_side, longer_side), color='black')
    padded_image.paste(image, (horizontal_padding, vertical_padding))

    # Resize the padded image to the target size with anti-aliasing
    padded_image_resized = padded_image.resize(target_size)

    return padded_image_resized


class RGBToGrayscaleLayer(keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, inputs):
        return tf.image.rgb_to_grayscale(inputs)


class GrayscaleToRGBLayer(keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, inputs):
        # Ensure input has a channel dimension
        if inputs.shape.rank == 3:  # Shape: (None, 224, 224)
            inputs = tf.expand_dims(inputs, axis=-1)  # Shape: (None, 224, 224, 1)
        return tf.image.grayscale_to_rgb(inputs)

    def compute_output_shape(self, input_shape):
        return input_shape[:-1] + (3,)  # Output has 3 channels

# Define ImageEnhancementLayer
class ImageEnhancementLayer(keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, inputs):
        if tf.executing_eagerly():
            # PIL-based enhancements (for debugging and visualization)
            img = tf.keras.preprocessing.image.array_to_img(inputs[0])  # Access first image in batch
            brightness_factor = 1 + (34 / 100)
            x_enhanced = ImageEnhance.Brightness(img).enhance(brightness_factor)
            contrast_factor = 1 + (30 / 100)
            x_enhanced = ImageEnhance.Contrast(x_enhanced).enhance(contrast_factor)
            sharpness_factor = 1 + (70 / 100)
            x_enhanced = ImageEnhance.Sharpness(x_enhanced).enhance(sharpness_factor)
            x_enhanced = x_enhanced.filter(ImageFilter.MedianFilter(size=1))

            # Convert back to tensor
            x_enhanced = tf.keras.preprocessing.image.img_to_array(x_enhanced)
            x_enhanced = tf.expand_dims(x_enhanced, 0)  # Restore batch dimension
            x_enhanced = tf.cast(x_enhanced, inputs.dtype)
        else:
            # Placeholder for TensorFlow-based enhancements during training
            x_enhanced = inputs
        return x_enhanced

class StrokeWidthLayer(layers.Layer):
    def call(self, sobel_output):
        # Ensure the input is from the SobelLayer
        if sobel_output.shape[-1] != 1:
            raise ValueError("Input must be the output of SobelLayer with 1 channel.")

        # Calculate the stroke width from the Sobel output
        # Here, we can compute the average or apply some additional processing
        stroke_width = tf.reduce_mean(sobel_output, axis=-1, keepdims=True)  # Example calculation

        # Normalize the stroke width
        normalized_stroke_width = stroke_width / (tf.reduce_max(stroke_width) + 1e-8)

        return normalized_stroke_width  # Output shape: (batch_size, height, width, 1)

    def compute_output_shape(self, input_shape):
        return input_shape  # Ensure output shape matches input shape

# Define SobelLayer
class SobelLayer(keras.layers.Layer):
    def __init__(self, **kwargs):
        super(SobelLayer, self).__init__(**kwargs)

        # Define Sobel kernels as constants
        self.vertical_kernel = tf.constant([[-1.0, 0.0, 1.0],
                                            [-2.0, 0.0, 2.0],
                                            [-1.0, 0.0, 1.0]], dtype=tf.float32)

        self.horizontal_kernel = tf.constant([[1.0, 2.0, 1.0],
                                              [0.0, 0.0, 0.0],
                                              [-1.0, -2.0, -1.0]], dtype=tf.float32)

        # Reshape the kernels to be 4D tensors (height, width, input channels, output channels)
        self.vertical_kernel = self.vertical_kernel[:, :, tf.newaxis, tf.newaxis]
        self.horizontal_kernel = self.horizontal_kernel[:, :, tf.newaxis, tf.newaxis]

    def call(self, inputs):
        # Ensure input is a grayscale image
        if inputs.shape[-1] != 1:
            raise ValueError("Input must be a grayscale image with 1 channel.")

        # Convert inputs to float32
        inputs_float32 = tf.cast(inputs, tf.float32)

        # Apply vertical and horizontal Sobel filters using 2D convolution
        vertical_edges = tf.nn.conv2d(inputs_float32, self.vertical_kernel, strides=[1, 1, 1, 1], padding='SAME')
        horizontal_edges = tf.nn.conv2d(inputs_float32, self.horizontal_kernel, strides=[1, 1, 1, 1], padding='SAME')

        # Compute the magnitude of the gradients (L2 norm of vertical and horizontal gradients)
        gradient_magnitude = tf.sqrt(tf.square(vertical_edges) + tf.square(horizontal_edges))

        return tf.expand_dims(gradient_magnitude, axis=-1)  # Ensure output is (batch_size, height, width, 1)

    def compute_output_shape(self, input_shape):
        return input_shape


class RobertsLayer(layers.Layer):
    def call(self, inputs):
        # Convert inputs to float32
        inputs_float32 = tf.cast(inputs, tf.float32)

        # Check the number of channels
        num_channels = inputs.shape[-1]

        # Define Roberts Cross kernels
        kernel_x = tf.constant([[1, 0], [0, -1]], dtype=tf.float32)
        kernel_y = tf.constant([[0, 1], [-1, 0]], dtype=tf.float32)

        # Reshape kernels to match the number of input channels
        kernel_x = tf.reshape(kernel_x, [2, 2, 1, 1])
        kernel_y = tf.reshape(kernel_y, [2, 2, 1, 1])

        # Apply convolution using Roberts Cross kernels for each channel
        grad_x = [tf.nn.conv2d(inputs_float32[..., i:i+1], kernel_x, strides=[1, 1, 1, 1], padding='SAME') for i in range(num_channels)]
        grad_y = [tf.nn.conv2d(inputs_float32[..., i:i+1], kernel_y, strides=[1, 1, 1, 1], padding='SAME') for i in range(num_channels)]

        # Concatenate the gradients along the channel dimension (if more than one channel)
        grad_x = tf.concat(grad_x, axis=-1)
        grad_y = tf.concat(grad_y, axis=-1)

        # Compute the gradient magnitude
        grad_magnitude = tf.sqrt(tf.square(grad_x) + tf.square(grad_y))

        # Normalize the gradient magnitudes
        grad_magnitude /= tf.reduce_max(grad_magnitude)

        # Return the result with the same number of channels as input
        return grad_magnitude

    def compute_output_shape(self, input_shape):
        return input_shape

class MorphologicalLayer(layers.Layer):
    def __init__(self, kernel_size=(2, 2), **kwargs):
        super(MorphologicalLayer, self).__init__(**kwargs)
        self.kernel_size = kernel_size

    def call(self, inputs):
        # Ensure inputs have the shape (batch_size, height, width, 1) for grayscale
        if inputs.shape[-1] != 1:
            raise ValueError("Input must be a grayscale image with a single channel.")

        # Define the kernel for dilation and erosion
        kernel = tf.ones(tuple(self.kernel_size) + (1,), dtype=tf.float32)

        # Perform dilation
        dilated_image = tf.image.extract_patches(
            images=inputs,
            sizes=[1, self.kernel_size[0], self.kernel_size[1], 1],
            strides=[1, 1, 1, 1],
            rates=[1, 1, 1, 1],
            padding='SAME'
        )
        dilated_image = tf.reduce_max(dilated_image, axis=-1, keepdims=True)

        # Perform erosion
        eroded_image = tf.image.extract_patches(
            images=dilated_image,
            sizes=[1, self.kernel_size[0], self.kernel_size[1], 1],
            strides=[1, 1, 1, 1],
            rates=[1, 1, 1, 1],
            padding='SAME'
        )
        eroded_image = tf.reduce_min(eroded_image, axis=-1, keepdims=True)
        return eroded_image

    def compute_output_shape(self, input_shape):
        return input_shape  # Ensure output shape matches input shape


class SqueezeLayer(layers.Layer):
    def __init__(self, axis=-1, **kwargs):
        super(SqueezeLayer, self).__init__(**kwargs)
        self.axis = axis

    def call(self, inputs):
        squeezed = tf.squeeze(inputs, axis=self.axis)  # Remove extra dimensions

        # If squeezed shape has 3 dimensions and we have grayscale images, restore the channel dimension
        if len(squeezed.shape) == 3:  # (batch, height, width) for grayscale image
            restored = tf.expand_dims(squeezed, axis=-1)  # (batch_size, 224, 224, 1)
        else:
            restored = squeezed  # No expansion needed, already 4D

        return restored


# Preprocessing function
def preprocess_image(img):
    img = pad_and_resize(img, target_size=image_size)
    img_array = np.array(img)
    if img_array.shape[-1] == 4:
        img_array = img_array[..., :3]
    img_array = np.expand_dims(img_array, 0) / 255.0
    return img_array

# vgg16 full processing pipeline
def vgg16_processing_pipeline(img_array):
    x = ImageEnhancementLayer()(img_array)
    x = RGBToGrayscaleLayer()(x)
    x = StrokeWidthLayer()(x)
    x = SqueezeLayer(axis=-1)(x)
    final_output = GrayscaleToRGBLayer()(x)

    return final_output

# vgg16_roberts full processing pipeline
def roberts_processing_pipeline(img_array):
    x = RGBToGrayscaleLayer()(img_array)
    x = RobertsLayer()(x)
    x = StrokeWidthLayer()(x)
    x = SqueezeLayer(axis=-1)(x)
    x = MorphologicalLayer()(x)
    final_output = GrayscaleToRGBLayer()(x)

    return final_output

# vgg16_sobel full processing pipeline
def sobel_processing_pipeline(img_array):
    x = RGBToGrayscaleLayer()(img_array)
    x = SobelLayer()(x)
    x = StrokeWidthLayer()(x)
    x = SqueezeLayer(axis=-1)(x)
    x = MorphologicalLayer()(x)
    final_output = GrayscaleToRGBLayer()(x)

    return final_output


def analyze_handwriting(img):
    try:
        # Convert image to grayscale for analysis
        img_gray = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2GRAY)

        # Apply adaptive thresholding to handle varying handwriting qualities
        binary_image = cv2.adaptiveThreshold(
            img_gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2
        )

        # Extract contours
        contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Debug: Print the number of contours found
        print(f"Number of contours found: {len(contours)}")

        if len(contours) == 0:
            return "No contours found in the image."

        # Extract bounding boxes
        bounding_boxes = [cv2.boundingRect(contour) for contour in contours]

        # Filter out invalid bounding boxes (e.g., zero width or height)
        bounding_boxes = [b for b in bounding_boxes if b[2] > 0 and b[3] > 0]

        # Debug: Print bounding boxes
        print(f"Bounding boxes: {bounding_boxes}")

        if len(bounding_boxes) == 0:
            return "No valid bounding boxes found."

        # Calculate Inter-Character Spacing
        spacings = [
            bounding_boxes[i + 1][0] - (bounding_boxes[i][0] + bounding_boxes[i][2])
            for i in range(len(bounding_boxes) - 1)
        ]
        avg_spacing = np.mean(spacings) if spacings else 0

        # Calculate Baseline Alignment
        baseline_y = [y + h for _, y, _, h in bounding_boxes]
        mean_baseline = np.mean(baseline_y) if baseline_y else 0
        baseline_deviations = [abs(y - mean_baseline) for y in baseline_y]
        avg_baseline_deviation = np.mean(baseline_deviations) if baseline_deviations else 0

        # Calculate Character Size Variation
        heights = [h for _, _, _, h in bounding_boxes]
        size_std_dev = np.std(heights) if heights else 0

        return avg_spacing, avg_baseline_deviation, size_std_dev

    except Exception as e:
        return f"Error in handwriting analysis: {e}"


def label_criteria(value, thresholds, severity='spacing'):
    if severity == 'spacing':
        # Assuming thresholds is a list with [min_spacing, max_spacing]
        if value < thresholds[0]:
            return "Severely Irregular Spacing"
        else:
            return "Normal to Slightly Irregular Spacing"

    elif severity == 'baseline':
        # Assuming thresholds is a single value for baseline threshold
        if value <= thresholds:
            return "Normal to Mild Misalignment"
        else:
            return "Severe Misalignment"

    elif severity == 'size':
        # Assuming thresholds is a list with [min_size, max_size]
        if value <= thresholds:
            return "Normal to Moderate Size Variation"
        else:
            return "Severe Size Variation"

    return "Invalid Threshold"

In [3]:
# Load the vgg16 model
with custom_object_scope({
    'ImageEnhancementLayer': ImageEnhancementLayer,
    'RGBToGrayscaleLayer': RGBToGrayscaleLayer,
    'StrokeWidthLayer': StrokeWidthLayer,
    'GrayscaleToRGBLayer': GrayscaleToRGBLayer}):
    vgg16_model = load_model("/content/drive/Shared drives/CS FILES/Model/vgg16_model.h5")


# Load the vgg16_roberts model
with keras.utils.custom_object_scope({
    'RGBToGrayscaleLayer': RGBToGrayscaleLayer,
    'RobertsLayer': RobertsLayer,
    'StrokeWidthLayer': StrokeWidthLayer,
    'SqueezeLayer': SqueezeLayer,
    'MorphologicalLayer': MorphologicalLayer,
    'GrayscaleToRGBLayer': GrayscaleToRGBLayer
}):
    roberts_model = load_model("/content/drive/Shared drives/CS FILES/Model/vgg16_roberts_model.h5")

# Load the vgg16_sobel model
with keras.utils.custom_object_scope({
    'RGBToGrayscaleLayer': RGBToGrayscaleLayer,
    'SobelLayer': SobelLayer,
    'StrokeWidthLayer': StrokeWidthLayer,
    'SqueezeLayer': SqueezeLayer,
    'MorphologicalLayer': MorphologicalLayer,
    'GrayscaleToRGBLayer': GrayscaleToRGBLayer
}):
    sobel_model = load_model("/content/drive/Shared drives/CS FILES/Model/vgg16_sobel_model5.h5")



# Define the class labels
class_labels = ["Low Potential Dysgraphia", "Potential Dysgraphia"]

# Diagnostic messages for each class
diagnostic_messages = {
    "Low Potential Dysgraphia": "It suggests a minimal impairment in writing; however, observing difficulty in hand grip and positioning may still be beneficial.",
    "Potential Dysgraphia": "It is highly recommended to consult a specialist for further evaluation and support."
}

# Define the image size (update based on your model's input size)
image_size = (224, 224)


# vgg16 prediction function
def vgg_predict_image(img):
    try:
        # Calculate handwriting metrics
        avg_spacing, avg_baseline_dev, size_std_dev = analyze_handwriting(img)

        # Classify the raw values
        spacing_class = label_criteria(avg_spacing, [-125, 26], severity='spacing')
        baseline_class = label_criteria(avg_baseline_dev, [57], severity='baseline')
        size_class = label_criteria(size_std_dev, [51], severity='size')  # or [20, 50] for two thresholds

         # Preprocess image
        img_array = preprocess_image(img)

        # Process the image through the full pipeline
        final_processed_image = vgg16_processing_pipeline(img_array)

        # Run model prediction
        predictions = vgg16_model.predict(img_array)
        predicted_class_index = np.argmax(predictions[0])
        class_label = class_labels[predicted_class_index]
        diagnostic_prompt = diagnostic_messages[class_label]

        # Calculate percentages
        low_potential_percentage = predictions[0][0] * 100
        potential_percentage = predictions[0][1] * 100
        # Construct the output
        classification_text = (
            f'<div style="text-align: center;">'
            f'<span style="font-size: 18px;">This handwriting is <b>"{class_label}"</b></span><br>'
            f'<span style="font-size: 15px;"><i>{diagnostic_prompt}</i></span><br><br>'
            f'<span style="font-size: 14px;"><b>Confidence Level:</b></span><br>'
            f'<span style="font-size: 14px;">({low_potential_percentage:.2f}% LPD, {potential_percentage:.2f}% PD)</span><br><br>'
            f'<span style="font-size: 14px;"><b>VGG16 Model Accuracy:</b></span> 75.51%<br>'
            f'<span style="font-size: 14px;"><b>VGG16 Model Precision:</b></span> 91.56%<br><br>'
            f'<span style="font-size: 15px;"><b><u>Criteria:</u></b></span><br>'
            f'<span style="font-size: 14px;"><b>• Inter-Character Spacing:</b></span> {spacing_class} ({avg_spacing:.2f})<br>'
            f'<span style="font-size: 14px;"><b>• Baseline Alignment:</b></span> {baseline_class} ({avg_baseline_dev:.2f})<br>'
            f'<span style="font-size: 14px;"><b>• Character Size Variation:</b></span> {size_class} ({size_std_dev:.2f})<br><br>'
            f'</div><br>'
        )


        return classification_text, final_processed_image

    except Exception as e:
        return f"Error in making prediction: {e}"


# vgg16_roberts prediction function
def roberts_predict_image(img):
    try:
        # Calculate handwriting metrics
        avg_spacing, avg_baseline_dev, size_std_dev = analyze_handwriting(img)

        # Classify the raw values
        spacing_class = label_criteria(avg_spacing, [-125, 26], severity='spacing')
        baseline_class = label_criteria(avg_baseline_dev, [57], severity='baseline')
        size_class = label_criteria(size_std_dev, [51], severity='size')  # or [20, 50] for two thresholds

         # Preprocess image
        img_array = preprocess_image(img)

        # Process the image through the full pipeline
        final_processed_image = roberts_processing_pipeline(img_array)

        # Run model prediction
        predictions = roberts_model.predict(img_array)
        predicted_class_index = np.argmax(predictions[0])
        class_label = class_labels[predicted_class_index]
        diagnostic_prompt = diagnostic_messages[class_label]

        # Calculate percentages
        low_potential_percentage = predictions[0][0] * 100
        potential_percentage = predictions[0][1] * 100

        # Construct the output
        classification_text = (
            f'<div style="text-align: center;">'
            f'<span style="font-size: 18px;">This handwriting is <b>"{class_label}"</b></span><br>'
            f'<span style="font-size: 15px;"><i>{diagnostic_prompt}</i></span><br><br>'
            f'<span style="font-size: 14px;"><b>Confidence Level:</b></span><br>'
            f'<span style="font-size: 14px;">({low_potential_percentage:.2f}% LPD, {potential_percentage:.2f}% PD)</span><br><br>'
            f'<span style="font-size: 14px;"><b>VGG16 with Roberts Model Accuracy:</b> 81.88<br>'
            f'<span style="font-size: 14px;"><b>VGG16 with Roberts Model Precision:</b> 93.59%<br><br>'
            f'<span style="font-size: 15px;"><b><u>Criteria:</u></b></span><br>'
            f'<span style="font-size: 14px;"><b>• Inter-Character Spacing:</b></span> {spacing_class} ({avg_spacing:.2f})<br>'
            f'<span style="font-size: 14px;"><b>• Baseline Alignment:</b></span> {baseline_class} ({avg_baseline_dev:.2f})<br>'
            f'<span style="font-size: 14px;"><b>• Character Size Variation:</b></span> {size_class} ({size_std_dev:.2f})<br><br>'
            f'</div><br>'
        )


        return classification_text, final_processed_image

    except Exception as e:
        return f"Error in making prediction: {e}"


# vgg16_sobel prediction function
def sobel_predict_image(img):
    try:
        # Calculate handwriting metrics
        avg_spacing, avg_baseline_dev, size_std_dev = analyze_handwriting(img)

        # Classify the raw values
        spacing_class = label_criteria(avg_spacing, [-125, 26], severity='spacing')
        baseline_class = label_criteria(avg_baseline_dev, [57], severity='baseline')
        size_class = label_criteria(size_std_dev, [51], severity='size')  # or [20, 50] for two thresholds

         # Preprocess image
        img_array = preprocess_image(img)

        # Process the image through the full pipeline
        final_processed_image = sobel_processing_pipeline(img_array)

        # Run model prediction
        predictions = sobel_model.predict(img_array)
        predicted_class_index = np.argmax(predictions[0])
        class_label = class_labels[predicted_class_index]
        diagnostic_prompt = diagnostic_messages[class_label]

        # Calculate percentages
        low_potential_percentage = predictions[0][0] * 100
        potential_percentage = predictions[0][1] * 100

        # Construct the output
        classification_text = (
            f'<div style="text-align: center;">'
            f'<span style="font-size: 18px;">This handwriting is <b>"{class_label}"</b></span><br>'
            f'<span style="font-size: 15px;"><i>{diagnostic_prompt}</i></span><br><br>'
            f'<span style="font-size: 14px;"><b>Confidence Level:</b></span><br>'
            f'<span style="font-size: 14px;">({low_potential_percentage:.2f}% LPD, {potential_percentage:.2f}% PD)</span><br><br>'
            f'<span style="font-size: 14px;"><b>VGG16 with Sobel Model Accuracy:</b> 80.95%<br>'
            f'<span style="font-size: 14px;"><b>VGG16 with Sobel Model Precision:</b> 93.06%<br><br>'
            f'<span style="font-size: 15px;"><b><u>Criteria:</u></b></span><br>'
            f'<span style="font-size: 14px;"><b>• Inter-Character Spacing:</b></span> {spacing_class} ({avg_spacing:.2f})<br>'
            f'<span style="font-size: 14px;"><b>• Baseline Alignment:</b></span> {baseline_class} ({avg_baseline_dev:.2f})<br>'
            f'<span style="font-size: 14px;"><b>• Character Size Variation:</b></span> {size_class} ({size_std_dev:.2f})<br><br>'
            f'</div><br>'
        )


        return classification_text, final_processed_image

    except Exception as e:
        return f"Error in making prediction: {e}"

In [4]:

# vgg16 gradio  interface
def vgg16_gradio_interface(image):
    vgg_classification_text, final_processed_image = vgg_predict_image(image)
    vgg_processed_image = Image.fromarray((tf.squeeze(final_processed_image).numpy() * 255).astype(np.uint8))
    return vgg_processed_image, vgg_classification_text

# vgg16_roberts gradio  interface
def roberts_gradio_interface(image):
    roberts_classification_text, final_processed_image = roberts_predict_image(image)
    roberts_processed_image = Image.fromarray((tf.squeeze(final_processed_image).numpy() * 255).astype(np.uint8))
    return roberts_processed_image, roberts_classification_text

# vgg16_sobel gradio  interface
def sobel_gradio_interface(image):
    sobel_classification_text, final_processed_image = sobel_predict_image(image)
    sobel_processed_image = Image.fromarray((tf.squeeze(final_processed_image).numpy() * 255).astype(np.uint8))
    return sobel_processed_image, sobel_classification_text


with gr.Blocks(fill_height=True) as demo:
    gr.HTML("<h1 style='text-align: center; font-size: 30x;'>Dysgraphia Classification using VGG16 Architecture with Sobel and Roberts Edge Detection in an Offline Handwriting Analysis</h1><h2 style='text-align: center; font-size: 20px;'>Choose a model then input an image to test if it's Low Potential or  Potential Dysgraphia</h2>")
    with gr.Tab("vgg16"):
        with gr.Row():
            with gr.Column():
                vgg_input_image = gr.Image(type="pil", label="Input Image")
                vgg16_button = gr.Button("Classify")
            with gr.Column():
                vgg_processed_image = gr.Image(type="pil", label="Processed Image")
        with gr.Row():
            vgg_classification_text = gr.HTML(label="Classification Text")
    with gr.Tab("vgg16_roberts"):
        with gr.Row():
            with gr.Column():
                roberts_input_image = gr.Image(type="pil", label="Input Image")
                roberts_button = gr.Button("Classify")
            with gr.Column():
                roberts_processed_image = gr.Image(type="pil", label="Processed Image")
        with gr.Row():
            roberts_classification_text = gr.HTML(label="Classification Text")
    with gr.Tab("vgg16_sobel"):
        with gr.Row():
            with gr.Column():
                sobel_input_image = gr.Image(type="pil", label="Input Image")
                sobel_button = gr.Button("Classify")
            with gr.Column():
                sobel_processed_image = gr.Image(type="pil", label="Processed Image")
        with gr.Row():
            sobel_classification_text = gr.HTML(label="Classification Text")

    vgg16_button.click(
        fn=vgg16_gradio_interface,
        inputs=vgg_input_image,
        outputs=[vgg_processed_image, vgg_classification_text],
    )
    roberts_button.click(
        fn=roberts_gradio_interface,
        inputs=roberts_input_image,
        outputs=[roberts_processed_image, roberts_classification_text],
    )
    sobel_button.click(
        fn=sobel_gradio_interface,
        inputs=sobel_input_image,
        outputs=[sobel_processed_image, sobel_classification_text],
    )

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://87ebea027abdd4f969.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
